In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['training.csv', 'validation.csv']


In [2]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer


### Loading The Data

In [3]:
df_train = pd.read_csv('../input/training.csv',sep = ';',decimal=",")
pd.set_option('display.max_columns', 30)
print(df_train.head())

print(df_train.info())


  variable1  variable2  variable3 variable4 variable5 variable6 variable7  \
0         a      17.92   0.000054         u         g         c         v   
1         b      16.92   0.000034         y         p         k         v   
2         b      31.25   0.000112         u         g        ff        ff   
3         a      48.17   0.000133         u         g         i         o   
4         b      32.33   0.000350         u         g         k         v   

   variable8 variable9 variable10  variable11 variable12 variable13  \
0      1.750         f          t           1          t          g   
1      0.290         f          f           0          f          s   
2      0.000         f          t           1          f          g   
3      0.335         f          f           0          f          g   
4      0.500         f          f           0          t          g   

   variable14  variable15  variable17 variable18  variable19 classLabel  
0        80.0           5    800000.

In [4]:
df_test = pd.read_csv('../input/validation.csv',sep = ';',decimal=",")
pd.set_option('display.max_columns', 30)
print(df_test.head())
print(df_test.info())



  variable1  variable2  variable3 variable4 variable5 variable6 variable7  \
0         b      32.33   0.000750         u         g         e        bb   
1         b      23.58   0.000179         u         g         c         v   
2         b      36.42   0.000075         y         p         d         v   
3         b      18.42   0.001041         y         p        aa         v   
4         b      24.50   0.001334         y         p        aa         v   

   variable8 variable9 variable10  variable11 variable12 variable13  \
0      1.585         t          f           0          t          s   
1      0.540         f          f           0          t          g   
2      0.585         f          f           0          f          g   
3      0.125         t          f           0          f          g   
4      0.040         f          f           0          t          g   

   variable14  variable15  variable17 variable18  variable19 classLabel  
0       420.0           0   4200000.

## Understanding the Data :

Variable18 seems to have a lot of missing data 

We are Going to Test 2 approches : 

    First : ignore the column and build a model without it 
    Second : Use a simple imputation to try and fill the missing values then build our model
    
In both cases we are going to drop the missing values of the rest of the columns , as they are very small (less than 1% of the data )    

## First Approach : Ignore variable18 and build a model without it

In [5]:
df_train_min = df_train[df_train.columns.difference(['variable18'])]
df_train_min.dropna(how='any', inplace=True)
df_train_min.reset_index(inplace = True)
df_train_min = df_train_min[df_train_min.columns.difference(['index'])]

print(df_train_min.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 18 columns):
classLabel    3522 non-null object
variable1     3522 non-null object
variable10    3522 non-null object
variable11    3522 non-null int64
variable12    3522 non-null object
variable13    3522 non-null object
variable14    3522 non-null float64
variable15    3522 non-null int64
variable17    3522 non-null float64
variable19    3522 non-null int64
variable2     3522 non-null float64
variable3     3522 non-null float64
variable4     3522 non-null object
variable5     3522 non-null object
variable6     3522 non-null object
variable7     3522 non-null object
variable8     3522 non-null float64
variable9     3522 non-null object
dtypes: float64(5), int64(3), object(10)
memory usage: 495.4+ KB
None


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
df_test_min = df_test[df_test.columns.difference(['variable18'])]
df_test_min.dropna(how='any', inplace=True)
df_test_min.reset_index(inplace=True )
df_test_min = df_test_min[df_test_min.columns.difference(['index'])]

print(df_test_min.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 18 columns):
classLabel    191 non-null object
variable1     191 non-null object
variable10    191 non-null object
variable11    191 non-null int64
variable12    191 non-null object
variable13    191 non-null object
variable14    191 non-null float64
variable15    191 non-null int64
variable17    191 non-null float64
variable19    191 non-null int64
variable2     191 non-null float64
variable3     191 non-null float64
variable4     191 non-null object
variable5     191 non-null object
variable6     191 non-null object
variable7     191 non-null object
variable8     191 non-null float64
variable9     191 non-null object
dtypes: float64(5), int64(3), object(10)
memory usage: 26.9+ KB
None


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
dftm = df_train_min.append(df_test_min,ignore_index = True)
print(dftm.head())
print(dftm.info())

  classLabel variable1 variable10  variable11 variable12 variable13  \
0        no.         a          t           1          t          g   
1        no.         b          f           0          f          s   
2        no.         b          t           1          f          g   
3        no.         a          f           0          f          g   
4        no.         b          f           0          t          g   

   variable14  variable15  variable17  variable19  variable2  variable3  \
0        80.0           5    800000.0           0      17.92   0.000054   
1       200.0           0   2000000.0           0      16.92   0.000034   
2        96.0          19    960000.0           0      31.25   0.000112   
3         0.0         120         0.0           0      48.17   0.000133   
4       232.0           0   2320000.0           0      32.33   0.000350   

  variable4 variable5 variable6 variable7  variable8 variable9  
0         u         g         c         v      1.750     

In [8]:
cat_list = dftm[['variable1','variable4','variable5','variable6','variable7','variable9','variable10',
         'variable12','variable13','classLabel']]
for i in cat_list : 
    print('For col named %s :' % i,'\n',dftm[i].value_counts())

For col named variable1 : 
 b    2430
a    1283
Name: variable1, dtype: int64
For col named variable4 : 
 u    3094
y     587
l      32
Name: variable4, dtype: int64
For col named variable5 : 
 g     3094
p      587
gg      32
Name: variable5, dtype: int64
For col named variable6 : 
 c     793
q     600
W     378
cc    355
x     321
aa    232
i     205
m     188
k     183
e     144
ff    140
d     101
j      40
r      33
Name: variable6, dtype: int64
For col named variable7 : 
 v     2046
h     1007
bb     338
ff     159
z       53
j       38
n       34
dd      21
o       17
Name: variable7, dtype: int64
For col named variable9 : 
 t    3199
f     514
Name: variable9, dtype: int64
For col named variable10 : 
 t    2252
f    1461
Name: variable10, dtype: int64
For col named variable12 : 
 f    1866
t    1847
Name: variable12, dtype: int64
For col named variable13 : 
 g    3493
s     203
p      17
Name: variable13, dtype: int64
For col named classLabel : 
 yes.    3356
no.      357
Name:

In [9]:
#one hot vector
dftm_ohv = pd.get_dummies(dftm,columns=['variable1','variable4','variable5','variable6','variable7','variable9','variable10',
         'variable12','variable13','classLabel'],drop_first =True)
print(dftm_ohv.head())

   variable11  variable14  variable15  variable17  variable19  variable2  \
0           1        80.0           5    800000.0           0      17.92   
1           0       200.0           0   2000000.0           0      16.92   
2           1        96.0          19    960000.0           0      31.25   
3           0         0.0         120         0.0           0      48.17   
4           0       232.0           0   2320000.0           0      32.33   

   variable3  variable8  variable1_b  variable4_u  variable4_y  variable5_gg  \
0   0.000054      1.750            0            1            0             0   
1   0.000034      0.290            1            0            1             0   
2   0.000112      0.000            1            1            0             0   
3   0.000133      0.335            0            1            0             0   
4   0.000350      0.500            1            1            0             0   

   variable5_p  variable6_aa  variable6_c       ...         va

## Choice Of Models :
#### The Best Models are the RandomForestClassifier and BernoulliNB since they both perform well on categorical data 
#### Also , i am using class weights to try to overcome the Unblalned data since there's very high unbalance between the target labels


##### Building The First Models

In [10]:
dftm_ohv = dftm_ohv.astype(float)

x = dftm_ohv.drop('classLabel_yes.',axis = 1)
y = dftm_ohv['classLabel_yes.']

print(type(y))
x_train = x.iloc[0:3521,:].values
y_train = y.iloc[0:3521].values

x_test = x.iloc[3521:,:].values
y_test = y.iloc[3521:].values

<class 'pandas.core.series.Series'>


In [11]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


In [12]:

class_weight_list = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weight = dict(zip(np.unique(y_train), class_weight_list))

print(class_weight)

{0.0: 6.823643410852713, 1.0: 0.5395341710082746}


In [13]:

rfc = RandomForestClassifier(n_estimators=1,max_depth=5,class_weight=class_weight,random_state=0)

param_grid = {'n_estimators':[1,2,3],'max_depth':range(3,12)}

gs = GridSearchCV(rfc, param_grid=param_grid,scoring='f1',cv=10)
gs.fit(x_train,y_train)
y_pred = gs.predict(x_test)
print(gs.best_estimator_)
print("Best score: %0.3f" % gs.best_score_)

from sklearn.metrics import classification_report
print(gs.score(x_test,y_test))
print(classification_report(y_test, y_pred))


RandomForestClassifier(bootstrap=True,
            class_weight={0.0: 6.823643410852713, 1.0: 0.5395341710082746},
            criterion='gini', max_depth=11, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
Best score: 1.000
0.5027322404371585
              precision    recall  f1-score   support

         0.0       0.54      0.56      0.55        99
         1.0       0.51      0.49      0.50        93

   micro avg       0.53      0.53      0.53       192
   macro avg       0.53      0.53      0.52       192
weighted avg       0.53      0.53      0.53       192



In [14]:
from sklearn.naive_bayes import BernoulliNB
ber = BernoulliNB()
ber.fit(x_train,y_train)
y_pred=ber.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.65      0.48      0.55        99
         1.0       0.57      0.72      0.64        93

   micro avg       0.60      0.60      0.60       192
   macro avg       0.61      0.60      0.59       192
weighted avg       0.61      0.60      0.59       192



## Second Approach : Impute Variable18 then build the model 

In [15]:
dftotal = df_train.append(df_test)
dftotal = dftotal.dropna(subset=['variable1', 'variable2','variable4','variable5','variable6','variable7','variable14','variable17']).reset_index()
print(dftotal.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713 entries, 0 to 3712
Data columns (total 20 columns):
index         3713 non-null int64
variable1     3713 non-null object
variable2     3713 non-null float64
variable3     3713 non-null float64
variable4     3713 non-null object
variable5     3713 non-null object
variable6     3713 non-null object
variable7     3713 non-null object
variable8     3713 non-null float64
variable9     3713 non-null object
variable10    3713 non-null object
variable11    3713 non-null int64
variable12    3713 non-null object
variable13    3713 non-null object
variable14    3713 non-null float64
variable15    3713 non-null int64
variable17    3713 non-null float64
variable18    1547 non-null object
variable19    3713 non-null int64
classLabel    3713 non-null object
dtypes: float64(5), int64(4), object(11)
memory usage: 580.2+ KB
None


In [16]:
#One Hot Vector 
dftotal = pd.get_dummies(dftotal,columns=['variable1','variable4','variable5','variable6','variable7','variable9','variable10',
         'variable12','variable13','classLabel'],drop_first =True)
dftotal['variable18'] = dftotal['variable18'].replace({'t':'1','f':'0'}).astype('category')
dftotal = dftotal[dftotal.columns.difference(['index'])]

print(dftotal.head())
print(dftotal.info())

   classLabel_yes.  variable10_t  variable11  variable12_t  variable13_p  \
0                0             1           1             1             0   
1                0             0           0             0             0   
2                0             1           1             0             0   
3                0             0           0             0             0   
4                0             0           0             1             0   

   variable13_s  variable14  variable15  variable17 variable18  variable19  \
0             0        80.0           5    800000.0          1           0   
1             1       200.0           0   2000000.0        NaN           0   
2             0        96.0          19    960000.0          1           0   
3             0         0.0         120         0.0        NaN           0   
4             0       232.0           0   2320000.0          0           0   

   variable1_b  variable2  variable3  variable4_u     ...       variable6_

##### Impute Variable18

In [17]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent")

dfemp = pd.DataFrame(imp.fit_transform(dftotal),columns =dftotal.columns )
print(dfemp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713 entries, 0 to 3712
Data columns (total 41 columns):
classLabel_yes.    3713 non-null object
variable10_t       3713 non-null object
variable11         3713 non-null object
variable12_t       3713 non-null object
variable13_p       3713 non-null object
variable13_s       3713 non-null object
variable14         3713 non-null object
variable15         3713 non-null object
variable17         3713 non-null object
variable18         3713 non-null object
variable19         3713 non-null object
variable1_b        3713 non-null object
variable2          3713 non-null object
variable3          3713 non-null object
variable4_u        3713 non-null object
variable4_y        3713 non-null object
variable5_gg       3713 non-null object
variable5_p        3713 non-null object
variable6_aa       3713 non-null object
variable6_c        3713 non-null object
variable6_cc       3713 non-null object
variable6_d        3713 non-null object
variable6_e  

##### Build second Models

In [18]:
dfemp_ohv = dfemp.astype(float)

x = dfemp_ohv.drop('classLabel_yes.',axis = 1)
y = dfemp_ohv['classLabel_yes.']

print(type(y))
x_train = x.iloc[0:3521,:].values
y_train = y.iloc[0:3521].values

x_test = x.iloc[3521:,:].values
y_test = y.iloc[3521:].values

<class 'pandas.core.series.Series'>


In [19]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)



class_weight_list = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weight = dict(zip(np.unique(y_train), class_weight_list))

print(class_weight)

{0.0: 6.823643410852713, 1.0: 0.5395341710082746}


In [20]:
rfc = RandomForestClassifier(class_weight=class_weight,random_state=0)

param_grid = {'n_estimators':[1,2,3],'max_depth':range(3,12)}

gs = GridSearchCV(rfc, param_grid=param_grid,scoring='f1',cv=10)
gs.fit(x_train,y_train)
y_pred = gs.predict(x_test)
print(gs.best_estimator_)
print("Best score: %0.3f" % gs.best_score_)

from sklearn.metrics import classification_report
print(gs.score(x_test,y_test))
print(classification_report(y_test, y_pred))


RandomForestClassifier(bootstrap=True,
            class_weight={0.0: 6.823643410852713, 1.0: 0.5395341710082746},
            criterion='gini', max_depth=10, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
Best score: 1.000
0.4968944099378882
              precision    recall  f1-score   support

         0.0       0.57      0.72      0.64        99
         1.0       0.59      0.43      0.50        93

   micro avg       0.58      0.58      0.58       192
   macro avg       0.58      0.57      0.57       192
weighted avg       0.58      0.58      0.57       192



In [21]:
ber = BernoulliNB()
ber.fit(x_train,y_train)
y_pred=ber.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.51      0.57        99
         1.0       0.58      0.73      0.65        93

   micro avg       0.61      0.61      0.61       192
   macro avg       0.62      0.62      0.61       192
weighted avg       0.63      0.61      0.61       192



###  The imputation of variable 18 made a good result regarding the overall f1 score , precision and recall , compared to dropping that vairable 